In [1]:
import numpy
import time
import cv2
from tkinter import *
from tkinter import simpledialog
# for beep
# from pygame import mixer
# import time
import xlwt;
from datetime import datetime;
from xlrd import open_workbook;
from xlwt import Workbook;
from xlutils.copy import copy
from pathlib import Path

In [2]:
import os
import cv2
import face_recognition
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from imutils.video import VideoStream
from keras.models import Sequential,model_from_json,Model
from scipy.ndimage import imread
from scipy.misc import imresize, imsave
import matplotlib.pyplot as plt
from PIL import Image
from playsound import playsound
IMG_SIZE = 24

C:\Users\vivek\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\vivek\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\vivek\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\vivek\Anaconda3\lib\site-packages\tensorflow\python\framework\

In [3]:
def hel():
    help(cv2)

def Contri():
    messagebox.showinfo("Contributors","\n1.Kartik Tyagi\n2. Naman Tyagi \n3. Abhay Mudgal \n")

def anotherWin():
    messagebox.showinfo("About",'Smart Attendance System v1.0\n Made Using\n-OpenCV\n-Numpy\n-Tkinter\n In Python 3')

In [4]:
def output(filename, sheet,num, name, present):
    my_file = Path('firebase/attendance_files/'+filename+str(datetime.now().date())+'.xls');
    if my_file.is_file():
        rb = open_workbook('firebase/attendance_files/'+filename+str(datetime.now().date())+'.xls');
        book = copy(rb);
        sh = book.get_sheet(0)
        # file exists
    else:
        book = xlwt.Workbook()
        sh = book.add_sheet(sheet)
    style0 = xlwt.easyxf('font: name Times New Roman, color-index red, bold on',
                         num_format_str='#,##0.00')
    style1 = xlwt.easyxf(num_format_str='D-MMM-YY')

    #variables = [x, y, z]
    #x_desc = 'Display'
    #y_desc = 'Dominance'
    #z_desc = 'Test'
    #desc = [x_desc, y_desc, z_desc]
    sh.write(0,0,datetime.now().date(),style1);


    col1_name = 'Name'
    col2_name = 'Present'


    sh.write(1,0,col1_name,style0);
    sh.write(1, 1, col2_name,style0);

    sh.write(num+1,0,name);
    sh.write(num+1, 1, present);

    fullname=filename+str(datetime.now().date())+'.xls';
    book.save('firebase/attendance_files/'+fullname)
    #return fullname;

def predict(img, model):
    img = Image.fromarray(img, 'RGB').convert('L')
    img = imresize(img, (IMG_SIZE,IMG_SIZE)).astype('float32')
    img /= 255
    #plt.imshow(img,cmap = plt.cm.gray)
    img = img.reshape(1,IMG_SIZE,IMG_SIZE,1)
    
    prediction = model.predict(img)
    if prediction < 0.15:
        prediction = 'closed'
    else:
        prediction = 'open'
        
    return prediction

def load_model():
    json_file = open('weights/model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("weights/model.h5")
    return loaded_model
    
def init():
    # face_cascPath = 'haarcascade_frontalface_alt.xml'
    face_cascPath = 'lbpcascade_frontalface.xml'

    open_eye_cascPath = 'haarcascade_eye_tree_eyeglasses.xml'
    left_eye_cascPath = 'haarcascade_lefteye_2splits.xml'
    right_eye_cascPath ='haarcascade_righteye_2splits.xml'
    dataset = 'faces'

    face_detector = cv2.CascadeClassifier(face_cascPath)
    open_eyes_detector = cv2.CascadeClassifier(open_eye_cascPath)
    left_eye_detector = cv2.CascadeClassifier(left_eye_cascPath)
    right_eye_detector = cv2.CascadeClassifier(right_eye_cascPath)

    print("[LOG] Opening webcam ...")
    video_capture = VideoStream(src=0).start()

    model = load_model()

    print("[LOG] Collecting images ...")
    images = []
    for direc, _, files in tqdm(os.walk(dataset)):
        for file in files:
            if file.endswith("jpg"):
                images.append(os.path.join(direc,file))
    return (model,face_detector, open_eyes_detector, left_eye_detector,right_eye_detector, video_capture, images) 

def process_and_encode(images):
    # initialize the list of known encodings and known names
    known_encodings = []
    known_names = []
    print("[LOG] Encoding faces ...")

    for image_path in tqdm(images):
        # Load image
        image = cv2.imread(image_path)
        # Convert it from BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
     
        # detect face in the image and get its location (square boxes coordinates)
        boxes = face_recognition.face_locations(image, model='hog')
        # Encode the face into a 128-d embeddings vector
        encoding = face_recognition.face_encodings(image, boxes)
        # the person's name is the name of the folder where the image comes from
        name = image_path.split(os.path.sep)[-1]
        name = name[0:len(name)-4].split('_')[1]

        if len(encoding) > 0 : 
            known_encodings.append(encoding[0])
            known_names.append(name)

    return {"encodings": known_encodings, "names": known_names}

def isBlinking(history, maxFrames):
    """ @history: A string containing the history of eyes status 
         where a '1' means that the eyes were closed and '0' open.
        @maxFrames: The maximal number of successive frames where an eye is closed """
    for i in range(maxFrames):
        pattern = '1' + '0'*(i+1) + '1'
        if pattern in history:
            return True
    return False

def detect_and_display(model, video_capture, face_detector, open_eyes_detector, left_eye_detector, right_eye_detector, data, eyes_detected):
        frame = video_capture.read()
        # resize the frame
        frame = cv2.resize(frame, (600, 500), fx=0.6, fy=0.6)

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Detect faces
        faces = face_detector.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(50, 50),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        # for each detected face
        for (x,y,w,h) in faces:
            # print(x,y,w,h)
            
            # Encode the face into a 128-d embeddings vector
            encoding = face_recognition.face_encodings(rgb, [(y, x+w, y+h, x)])[0]
            
            # Compare the vector with all known faces encodings
            matches = face_recognition.compare_faces(data["encodings"], encoding)

            # For now we don't know the person name
            name = "Unknown"

            # If there is at least one match:
            if True in matches:
                matchedIdxs = [i for (i, b) in enumerate(matches) if b]
                counts = {}
                for i in matchedIdxs:
                    name = data["names"][i]
                    counts[name] = counts.get(name, 0) + 1

                # determine the recognized face with the largest number of votes
                name = max(counts, key=counts.get)

            face = frame[y:y+h,x:x+w]
            gray_face = gray[y:y+h,x:x+w]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            eyes = []
            
            # Eyes detection
            # check first if eyes are open (with glasses taking into account)
            open_eyes_glasses = open_eyes_detector.detectMultiScale(
                gray_face,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(30, 30),
                flags = cv2.CASCADE_SCALE_IMAGE
            )
            # if open_eyes_glasses detect eyes then they are open 
            if len(open_eyes_glasses) == 2:
                eyes_detected[name]+='1'
                for (ex,ey,ew,eh) in open_eyes_glasses:
                    cv2.rectangle(face,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            
            # otherwise try detecting eyes using left and right_eye_detector
            # which can detect open and closed eyes                
            else:
                # separate the face into left and right sides
                left_face = frame[y:y+h, x+int(w/2):x+w]
                left_face_gray = gray[y:y+h, x+int(w/2):x+w]

                right_face = frame[y:y+h, x:x+int(w/2)]
                right_face_gray = gray[y:y+h, x:x+int(w/2)]

                # Detect the left eye
                left_eye = left_eye_detector.detectMultiScale(
                    left_face_gray,
                    scaleFactor=1.1,
                    minNeighbors=5,
                    minSize=(30, 30),
                    flags = cv2.CASCADE_SCALE_IMAGE
                )

                # Detect the right eye
                right_eye = right_eye_detector.detectMultiScale(
                    right_face_gray,
                    scaleFactor=1.1,
                    minNeighbors=5,
                    minSize=(30, 30),
                    flags = cv2.CASCADE_SCALE_IMAGE
                )

                eye_status = '1' # we suppose the eyes are open

                # For each eye check wether the eye is closed.
                # If one is closed we conclude the eyes are closed
                for (ex,ey,ew,eh) in right_eye:
                    color = (0,255,0)
                    pred = predict(right_face[ey:ey+eh,ex:ex+ew],model)
                    if pred == 'closed':
                        eye_status='0'
                        color = (0,0,255)
                    cv2.rectangle(right_face,(ex,ey),(ex+ew,ey+eh),color,2)
                    
                for (ex,ey,ew,eh) in left_eye:
                    color = (0,255,0)
                    pred = predict(left_face[ey:ey+eh,ex:ex+ew],model)
                    if pred == 'closed':
                        eye_status='0'
                        color = (0,0,255)
                    cv2.rectangle(left_face,(ex,ey),(ex+ew,ey+eh),color,2)
                eyes_detected[name] += eye_status

            # Each time, we check if the person has blinked
            # If yes, we display its name
            if isBlinking(eyes_detected[name],3):
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                # Display name
                y = y - 15 if y - 15 > 15 else y + 15
                cv2.putText(frame, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)
                output('attendance','class 1',1,name,'yes')

        return frame

In [5]:
def exitt():
    exit()

def web():
    capture =cv2.VideoCapture(0)
    while True:
        ret,frame=capture.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF ==ord('q'):
            break
    capture.release()
    cv2.destroyAllWindows() 

def startAttendance():
    (model, face_detector, open_eyes_detector,left_eye_detector,right_eye_detector, video_capture, images) = init()
    data = process_and_encode(images)
    #print(data)
    eyes_detected = defaultdict(str)
    
    while True:
        frame = detect_and_display(model, video_capture, face_detector, open_eyes_detector,left_eye_detector,right_eye_detector, data, eyes_detected)
        cv2.imshow("Face Liveness Detector", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()
    video_capture.stop()

def webdet():
    capture =cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier('lbpcascade_frontalface.xml')
    eye_glass = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')


    while True:
        ret, frame = capture.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray)
    

        for (x,y,w,h) in faces:
            font = cv2.FONT_HERSHEY_COMPLEX
            cv2.putText(frame,'Face',(x+w,y+h),font,1,(250,250,250),2,cv2.LINE_AA)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
        
            eye_g = eye_glass.detectMultiScale(roi_gray)
            for (ex,ey,ew,eh) in eye_g:
                cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

       
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
    capture.release()
    cv2.destroyAllWindows()

def alert():
    mixer.init()
    alert=mixer.Sound('beep-07.wav')
    alert.play()
    time.sleep(0.1)
    alert.play()   
    
def data_Add():
    sampleNum = 0
    user_name = simpledialog.askstring("Enter Name", "Enter User Name")
    #print(user_name)
    
    while user_name=="":
        simpledialog.messagebox.showwarning("Error", "Name Field cannot be left blank !")
        user_name = simpledialog.askstring("user_name", "Enter User Name")
    
    simpledialog.messagebox.showinfo("Ready", "Please look at the Webcam.\nTurn your head a little while capturing.\nPlease add just one face at a time. \n Click 'OK' when you're ready.")
    capture =cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier('lbpcascade_frontalface.xml')
    eye_glass = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')

    while True:
        ret, frame = capture.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray)


        for (x,y,w,h) in faces:
            font = cv2.FONT_HERSHEY_COMPLEX
            cv2.putText(frame,'Face-'+str(sampleNum),(x+w,y+h),font,1,(250,250,250),2,cv2.LINE_AA)
            sampleNum = sampleNum + 1  #increment sample num till 21
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            face_path = 'faces/User_'+user_name+'_'+str(sampleNum)+'.jpg'
            cv2.imwrite(str(face_path), roi_color) 
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            cv2.waitKey(100)

        cv2.imshow('faces', frame)  #show image while capturing
        cv2.waitKey(7000)
        if(sampleNum > 3 or (cv2.waitKey(1) & 0xff == ord('q'))): #4 sample is collected
            break   
    capture.release()
    playsound('sound.mp3')
    cv2.destroyAllWindows()
    
def attend():
    os.startfile(os.getcwd()+"/firebase/attendance_files/attendance"+str(datetime.now().date())+'.xls')

In [6]:
root=Tk()
root.geometry('500x570')
frame = Frame(root, relief=RIDGE, borderwidth=2)
frame.pack(fill=BOTH,expand=1)
root.title('Smart Cam')
frame.config(background='light blue')
label = Label(frame, text="Smart Attendance App",bg='light blue',font=('Times 35 bold'))
label.pack(side=TOP)

background_image = PhotoImage(file="demo.png")
background_image = background_image.subsample(1, 1)
background = Label( image=background_image, bd=0)#made changes Label(root,)
background.pack(side=TOP)

menu = Menu(root)
root.config(menu=menu)

subm1 = Menu(menu)
menu.add_cascade(label="Tools",menu=subm1)
subm1.add_command(label="Open CV Docs",command=hel)

subm2 = Menu(menu)
menu.add_cascade(label="About",menu=subm2)
subm2.add_command(label="Driver Cam",command=anotherWin)
subm2.add_command(label="Contributors",command=Contri)


but1=Button(background,padx=5,pady=5,width=39,bg='white',fg='black',relief=GROOVE,command=webdet,text='Open Cam',font=('helvetica 15 bold'))
but1.place(x=5,y=104)

but2=Button(background,padx=5,pady=5,width=39,bg='white',fg='black',relief=GROOVE,command=data_Add,text='Add Person',font=('helvetica 15 bold'))
but2.place(x=5,y=176)

but3=Button(background,padx=5,pady=5,width=39,bg='white',fg='black',relief=GROOVE,command=startAttendance,text='Start',font=('helvetica 15 bold'))
but3.place(x=5,y=250)

but4=Button(background,padx=5,pady=5,width=39,bg='white',fg='black',relief=GROOVE,command=attend,text='Attendance Sheet',font=('helvetica 15 bold'))
but4.place(x=5,y=323)

but5=Button(background,padx=5,pady=5,width=5,bg='white',fg='black',relief=GROOVE,text='EXIT',command=exitt,font=('helvetica 15 bold'))
but5.place(x=210,y=400)

In [ ]:
root.mainloop()

[LOG] Opening webcam ...








2it [00:00, 1382.21it/s]
  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

[LOG] Collecting images ...
[LOG] Encoding faces ...


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.27it/s]
C:\Users\vivek\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\vivek\Anaconda3\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-5-17936fd7280d>", line 106, in attend
    os.startfile(os.getcwd()+"/firebase/attendance_files/attendance"+str(datetime.now().date())+'.xls')
OSError: [WinError 1155] No application is associated with the specified file for this operation: 'C:\\Users\\vivek\\Downloads\\Smart Attendance System/firebase/attendance_files/attendance2020-08-07.xls'
